In [2]:
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
from jinja2 import Template
from pyne import data

ModuleNotFoundError: No module named 'pyne'

First we want the helium density in $10^24$ atoms per cc.

\begin{align}
PV&=nRT\\
\frac{n}{V}&=\frac{P}{RT}
\end{align}

In [1]:

R = 8.314 # j/k-mol
temp = [250 ,750] #celsius
pres = [6*10**6, 5.84*10**6] #pascals
hedensity = np.zeros(len(temp))


for i, t in enumerate(temp):
    hedensity[i] = ((pres[i])/((t+273.15)*R)) # mol/m^3 

hedensavg = np.sum(hedensity)/len(hedensity)*(6.022*10**-7)# 10^24/cc

print(hedensavg)

NameError: name 'np' is not defined

### Isotopic composition of the fuel

The enrichment of the UCO kernels in the TRISO particles is known.  Additionally, we know the molar contentrations of the three compounds that make UCO: $UO_2$ , $UC$, and $UC_{1.86}$.  The atomic mass of each element is given using the PYNE python library.  The steps are as follows:

\begin{align}
Mass_{UO_{2}} = (enrichment) * M_{235U} + (1-enrichment) * M_{238U} + (2 * M_{16O} \\
Mass_{UC} = (enrichment) * M_{235U} + (1-enrichment) * M_{238U} + M_{12C} \\
Mass_{UC_{1.86}} = (enrichment) * M_{235U} + (1-enrichment) * M_{238U} + 1.86 * M_{12C} \\
\end{align}

The total mass of the UCO is the sum of these masses.  Then the isotopic concentration of each element is calculated as follows:

\begin{align}
16O wt\% = \frac{2 * mol\%UO_2 * M_{16O}}{Mass_{UCO}} \\
235U wt\% = \frac{enrichment * M_{235U}}{Mass_{UCO}} \\
238U wt\% = \frac{(1-enrichment) * M_{238U}}{Mass_{UCO}} \\
12C wt\% = \frac{mol\%UC_{1.86}* 1.86 * M_{12C} + mol\%UC_{100} * M_{12C}}{Mass_{UCO}} \\
\end{align}

In [14]:
#calculate isotopic comp. of uranium isotopes for UCO
#data on properties of UCO from Nagley et. all "Fabrication..."
#update this using pyne

molUO2 = 0.714
molUC186 = 0.123
molUC100 = 0.164

mu235 = data.atomic_mass('u235')
mu238 = data.atomic_mass('u238')
mo16 = data.atomic_mass('o16')
mc12 = data.atomic_mass('c12')

uenrich = 0.155

massUO2 = (uenrich*mu235) + (1-uenrich)*mu238 + (2*mo16)
massUC186 = (uenrich*mu235) + (1-uenrich)*mu238 + 1.86*mc12 #actually, should this be 1.86, or .86?
massUC100 = (uenrich*mu235) + (1-uenrich)*mu238 + mc12

massUCO = molUO2*massUO2 + molUC86*massUC186 + molUC100*massUC100

o16wt = (2*molUO2*mo16)/(massUCO)
u235wt = (uenrich*mu235)/(massUCO)
u238wt = ((1-uenrich)*mu238)/(massUCO)
c12wt = (molUC186*1.86*mc12 + molUC100*mc12)/(massUCO)

print(u235wt, u238wt, o16wt, c12wt)

15.99491461957
0.1328474919641083 0.7624328682381832 0.08606433036904058 0.017760029068465768


In [11]:
molUO2 + molUC186 + molUC100


1.001

In order to find the core dimensions, I took an image of the reactor core given by X-energy in a presentation to the NRC.  
<br>
Then, I opened it in an image viewer that gave the cursor's coordinates in pixels, measured the reactor core features in pixels.  Knowing the outer core diameter, developed a scale between pixels and meters, and used that to determine the core geometry.

\begin{align}
scale (\frac{cm}{px}) = 100* \frac{RPV_D (m)}{RPV_D (px)}\\
Dimension (cm) = Dimension (px) * scale
\end{align}

In [11]:
core_dm = 4.88 # m
core_dpx = 69 #px
scale = core_dm/core_dpx *100 # cm/px

inner_corerad_px = 17.5
inner_corerad_cm = inner_corerad_px *scale
reflect_px = 13
reflect_cm = reflect_px *scale

print(2*(reflect_cm+inner_corerad_cm)) #can't be more than 488, which is the diameter including RPV
print()
print(inner_corerad_cm)
print(reflect_cm)
print(inner_corerad_cm+reflect_cm)

431.4202898550724

123.76811594202897
91.94202898550724
215.7101449275362


I found the midpoint of the cones on the top and bottom , and used the distance between these two midpoints as an adjusted height.  Then, I simplified by letting the reflector directly surround this region.

In [10]:
mid_bott_y = 228
mid_top_y = 91.5

adj_h_px = mid_bott_y - mid_top_y #px

adj_h_cm = adj_h_px*scale

print(adj_h_cm)

965.391304347826


### Creating the input file

The first step to creating the input file is to create the pebble and particle location input cards.  This can be done using serpent's built in particle dispersal routine, which takes the dimensions of the particle, the volume they are dispersed in, and the particle number or packing fraction, the universe the particles are in, and creates a plain text file, where each line is:

\begin{align}
x-coordinate\ \ y-coordinate\ \  z-coordinate\ \  universe
\end{align}

In [ ]:
#simple to read in a text file, but shouldnt store a variable tens or hundreds of thousands of elements long
# need to manipulate file without doing that (hopefully)